In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

N=16153

%pwd

'/home/cseadmin/data/cys/TrafficFlowModel/data_process'

geo

---

In [14]:
sz_graph_drive=nx.read_gpickle("../data/sz_graph_drive.pkl")
sz_line_graph_drive=nx.read_gpickle("../data/sz_line_graph_drive.pkl")

list(sz_line_graph_drive.nodes)[:10]

[(598153574, 2767949724, 0),
 (2767949724, 2768006811, 0),
 (2767949724, 2768006811, 1),
 (598153574, 297121337, 0),
 (297121337, 2320506360, 0),
 (297121337, 1824553511, 0),
 (598153603, 598153605, 0),
 (598153605, 598153609, 0),
 (685056781, 6383446647, 0),
 (6383446647, 6383446646, 0)]

In [11]:
df_roads=pd.DataFrame()

df_roads["geo_id"]=sz_line_graph_drive.nodes
df_roads=df_roads.reset_index(drop=True)

df_roads["type"]="LineString"

df_roads["coordinates"]=None

df_roads

,geo_id,type,coordinates
0,"(598153574, 2767949724, 0)",LineString,None
1,"(2767949724, 2768006811, 0)",LineString,None
2,"(2767949724, 2768006811, 1)",LineString,None
3,"(598153574, 297121337, 0)",LineString,None
4,"(297121337, 2320506360, 0)",LineString,None
...,...,...,...
84301,"(9148694973, 9148694974, 0)",LineString,None
84302,"(9148694975, 9148694973, 0)",LineString,None
84303,"(9148694975, 9148694973, 1)",LineString,None
84304,"(9148694975, 9148694976, 0)",LineString,None


In [19]:
highway_types=["motorway", "trunk", "primary", "secondary", "motorway_link", "trunk_link", "primary_link", "secondary_link"]
def get_coords(row):
    road=row["geo_id"]

    road_data_dict=sz_graph_drive.get_edge_data(road[0], road[1])[road[2]]

    if isinstance(road_data_dict["highway"], list):
        for road_type in road_data_dict["highway"]:
            if road_type not in highway_types:
                return row
    elif isinstance(road_data_dict["highway"], str):
        if road_data_dict["highway"] not in highway_types:
            return row

    if "geometry" not in road_data_dict:
        return row

    row["coordinates"]=road_data_dict["geometry"].coords

    return row

df_roads=df_roads.apply(get_coords, axis=1)

df_roads

,geo_id,type,coordinates
0,"(598153574, 2767949724, 0)",LineString,"((114.1742099, 22.5622634), (114.1744957, 22.5..."
1,"(2767949724, 2768006811, 0)",LineString,None
2,"(2767949724, 2768006811, 1)",LineString,"((114.1782142, 22.5595428), (114.1791269, 22.5..."
3,"(598153574, 297121337, 0)",LineString,"((114.1742099, 22.5622634), (114.174588, 22.56..."
4,"(297121337, 2320506360, 0)",LineString,"((114.2259536, 22.5668991), (114.2284345, 22.5..."
...,...,...,...
84301,"(9148694973, 9148694974, 0)",LineString,None
84302,"(9148694975, 9148694973, 0)",LineString,None
84303,"(9148694975, 9148694973, 1)",LineString,None
84304,"(9148694975, 9148694976, 0)",LineString,None


In [21]:
df_roads=df_roads.dropna()
df_roads=df_roads.reset_index(drop=True)
df_roads["geo_id"]=df_roads.index

df_roads

,geo_id,type,coordinates
0,0,LineString,"((114.1742099, 22.5622634), (114.1744957, 22.5..."
1,1,LineString,"((114.1782142, 22.5595428), (114.1791269, 22.5..."
2,2,LineString,"((114.1742099, 22.5622634), (114.174588, 22.56..."
3,3,LineString,"((114.2259536, 22.5668991), (114.2284345, 22.5..."
4,4,LineString,"((114.2259536, 22.5668991), (114.2284254, 22.5..."
...,...,...,...
16148,16148,LineString,"((114.0159237, 22.6483238), (114.015941, 22.64..."
16149,16149,LineString,"((114.0105371, 22.6496528), (114.0085801, 22.6..."
16150,16150,LineString,"((114.0122871, 22.6624932), (114.0114325, 22.6..."
16151,16151,LineString,"((114.0122368, 22.6624536), (114.0122989, 22.6..."


In [23]:
# df_roads.to_csv("../data/libcity_atom/sz_taxi.geo", index=False)

rel

---

In [2]:
tm0_recalc_0diag=np.load("../data/trans_matrix_recalc.npy")[0]

for i in range(len(tm0_recalc_0diag)):
    tm0_recalc_0diag[i][i]=0

In [4]:
rel=[]
rel_id_counter=0

for i in range(N):
    for j in range(N):
        if (tm0_recalc_0diag[i][j]>0):
            rel.append([rel_id_counter, "geo", i, j, 1])
            rel_id_counter+=1

rel_df=pd.DataFrame(rel, columns=["rel_id", "type", "origin_id", "destination_id", "link_weight"])

rel_df

,rel_id,type,origin_id,destination_id,link_weight
0,0,geo,0,1,1
1,1,geo,0,2,1
2,2,geo,0,3,1
3,3,geo,0,4,1
4,4,geo,0,5,1
...,...,...,...,...,...
1798811,1798811,geo,16152,16121,1
1798812,1798812,geo,16152,16124,1
1798813,1798813,geo,16152,16135,1
1798814,1798814,geo,16152,16144,1


In [5]:
# rel_df.to_csv("../data/libcity_atom/sz_taxi.rel", index=False)